In [ ]:
#개발 환경 만들기
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

In [ ]:
# 데이터 가져오기
!git clone https://github.com/yhlee1627/deeplearning.git
dataframe = read_csv('/content/deeplearning/corona_daily.csv', usecols=[1], engine='python', skipfooter=3) #secols=[3]는 사용할 데이터를 의미 사용할 데이터가 4번째 열에 있는 확진자 수이므로 3(파이썬은 0부터 시작하죠?)을 넣어줍니다. 마지막 engine='python'은 사용할 언어를 의미
print(dataframe)
dataset = dataframe.values
dataset = dataset.astype('float32')#지금 읽어온 데이터는 정수형 데이터이므로 정수형 데이터를 소수점 단위까지 나누기 위해서는 실수로 바꿔 줄 필요가 있다

Cloning into 'deeplearning'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (5/5), done.
     Inspected
0         1352
1         2097
2         2598
3         3110
4         4325
..         ...
107     820289
108     826437
109     839475
110     852876
111     868666

[112 rows x 1 columns]


In [ ]:
#데이터 정규화 및 분류하기
scaler = MinMaxScaler(feature_range=(0, 1))
#정규화 scaler 방식 사용 데이터를 정규화하는 범위를 0~1 사이의 값(feature_range=(0, 1))으로 결정
Dataset = scaler.fit_transform(dataset)
 #fit_transform 함수를 사용하여 데이터를 정규화 정규화한 데이터를 Dataset으로 정함
train_data, test_data = train_test_split(Dataset, test_size=0.2, shuffle=False)
#train_test_split 함수를 사용하여 전체 데이터를 훈련 데이터와 검증 데이터로 분류
#분류할 데이터(Dataset), 검증 데이터 비율(test_size=0.2), 추출하는 방법(shuffle=False)
print(len(train_data), len(test_data))


89 23


In [ ]:
#데이터 형태 바꾸기
def create_dataset(dataset, look_back):
    x_data = []
    y_data = []
    #배열([])을 사용하여 각각 x_data와 y_data를 넣을 수 있는 공간을 만듬
    for i in range(len(dataset)-look_back-1):
        data = dataset[i:(i+look_back), 0]
        x_data.append(data)
        y_data.append(dataset[i + look_back, 0])
    return np.array(x_data), np.array(y_data)

In [1]:
# 입력 데이터 생성하기
look_back = 3
x_train, y_train = create_dataset(train_data, look_back)
#create_dataset 함수를 호출
#첫 번째 인자에 훈련 데이터(train_data)를 두 번째 인자에 look_back
x_test, y_test = create_dataset(test_data, look_back)
#첫 번째 인자에 검증 데이터(test_data)를 넣습니다. 그리고 두 번째 인자에 look_back
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

NameError: name 'create_dataset' is not defined

In [ ]:
#인공지능 모델에 넣어줄 형태로 변환하기
X_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
#형태를 바꿔 주는 함수(reshape) 함수의 첫 번째에는 바꿀 데이터, 두 번째에는 어떤 형태로 바꿀지
X_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
print(X_train.shape)
print(X_test.shape)

(85, 1, 3)
(19, 1, 3)


In [ ]:
#인공지능 모델 만들기
model = Sequential()
#model이라는 인공지능 모델을 만들겠습니다. 이 모델은 시퀀셜(Sequential) 모델입니다.
model.add(SimpleRNN(3, input_shape=(1, look_back)))
#RNN 기법 중 SimpleRNN을 사용
model.add(Dense(1, activation="linear"))
#최종 예측 값(즉 여기선 확진자수 노드는 1개)
model.compile(loss='mse', optimizer='adam')
#인공지능을 계산하는 방법을 결정
# 손실 함수는 mse(평균 제곱 오차, mean_squared_error)로, 옵티마이저는 adam 옵티마이저를 사용
#다양한 손실 함수 중 왜 하필이면 평균 제곱 오차를 사용할까요? 바로 실제 확진자의 수와 예측한 값의 차이를 바탕으로 오차를 나타낼 수 있기 때문입니다.
model.summary()
#생성된 모델을 요약
# Model: "sequential_1"는 첫 번째 순서로 만든 순차 모델이라는 의미입니다. (실습 환경에 따라 숫자는 달라질 수 있습니다.)
#simple_rnn (Simple RNN)은 SimpleRNN을 사용하였다는 의미입니다
#Output Shape에서 볼 수 있듯이 총 노드의 수는 3개입니다.
#Param은 simple_rnn의 파라미터(가중치와 편향 값)의 수를 의미합니다.
#dense (Dense)는 출력층에서의 형태 및 파라미터 수를 의미합니다.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 3)                   │              21 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │               4 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25 (100.00 B)

 Trainable params: 25 (100.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#모델 학습시키기
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1)
#입력 데이터(x_train) 출력 데이터(y_train), 반복 횟수(epochs=100), 한 번에 학습시킬 데이터의 양(batch_size=1)을 설정

Epoch 1/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.1185
Epoch 2/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0227
Epoch 3/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0130
Epoch 4/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0077
Epoch 5/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0049
Epoch 6/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0024
Epoch 7/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0020
Epoch 8/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0011
Epoch 9/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0010
Epoch 10/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011
Epoch 11/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011
Epoch 12/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011
Epoch 13/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0012
Epoch 14/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.6388e-04
Epoch 15/100
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9.1375e

In [ ]:
# 데이터 예측하기
trainPredict = model.predict(X_train)
#생성한 인공지능 모델에 데이터를 넣어서 결괏값을 생성하는 predict 함수
#훈련 데이터(X_train)의 값을 모델에 넣어 값을 예측
#그 예측 값을 trainPredict에 넣음
testPredict = model.predict(X_test)
#검증 데이터(X_test)의 값을 모델에 넣어 값을 예측
#예측 값을 trainPredict에 넣음
TrainPredict = scaler.inverse_transform(trainPredict)
#생성한 인공지능 모델에 훈련 데이터를 넣어서 얻은 결과가 = trainPredic
#trainPredict 에 저장된 값은 0~1사이임 (정규화 과정으로 인해서)
#이를 자연수의 형태로 바꾸기 위해서는 훈련 데이터의 예측 값(trainPredict)을 scaler 라이브러리의 inverse_transform 함수를 사용
#이 함수를 사용하면 0과 1 사이의 값을 정규화하기 전의 확진자의 수로 바꿀 수 있음
Y_train = scaler.inverse_transform([y_train])
TestPredict = scaler.inverse_transform(testPredict)
Y_test = scaler.inverse_transform([y_test])

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [ ]:
#모델의 정확도 살펴보기
trainScore = math.sqrt(mean_squared_error(Y_train[0], TrainPredict[:,0]))
# mean_squared_error 함수를 사용
# 첫 번째에는 실제 정답값 전체를 가져오기 위해 Y_train[0]을
#두 번째에는 예측 값 전체를 가져오기 위해 TrainPredict[:,0]을 넣음
#예측 값의 형태가 [:,0]인 이유는 배열이 2차원 배열로 이루어져 있기 때문
#평균 제곱근 오차를 구하기 위해 결괏값을 제곱근한 값을 math.sqrt 함수를 사용
print('Train Score: %.2f RMSE' % (trainScore))
#실수값을 출력할 때에는 %f를 사용
#소수 둘째 자리까지만 출력하라는 의미로 %.2f를 입력
testScore = math.sqrt(mean_squared_error(Y_test[0], TestPredict[:,0]))
#평균 제곱 오차를 구할 때 훈련 데이터의 값((Y_test)과 그 예측 값(TestPredict)을 넣어줌
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 4929.09 RMSE
Test Score: 39999.47 RMSE


In [ ]:
#결과를 그래프로 확인하기
trainPredictPlot = np.empty_like(dataset)
#전체 데이터(dataset)와 동일한(np.empty_like) 형태의 넘파이 배열(trainPredictPlot)을 만들어 줌
trainPredictPlot[:, :] = np.nan
#만들어진 배열의 모든 값을 nan으로 설정
#콜론(:)은 모든값을 의미함. (처음):(마지막)에서 처음과 마지막은 생략하고 나타내는 것
trainPredictPlot[look_back:len(TrainPredict)+look_back, :] = TrainPredict
#처음 3일치는 건너뛰고 4일차(look_back)부터 TrainPredict 배열의 원소 수까지(len(TrainPredict)+look_back)를 범위로 지정
testPredictPlot = np.empty_like(dataset)
#전체 데이터(dataset)와 동일한(np.empty_like) 넘파이 배열(trainPredictPlot)을 만들어 줌
testPredictPlot[:, :] = np.nan
#만들어진 배열의 모든 값을 nan으로 설정
testPredictPlot[len(TrainPredict)+((look_back+1)*2):len(dataset), :] = TestPredict
#검증 데이터를 예측한 결괏값을 그래프로 나타내기 위한 코드
#검증 데이터의 예측 값을 넣는 시작점은 훈련 데이터를 예측한 값 이후(len(TrainPredict)+look_back+1)에 3일치의 예측 값을 건너뛴(look_back+1) 자리임
plt.plot(dataset)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show( )